## **Use Case Overview**

Imagine you’re part of an R&D team that needs to merge structured data (e.g., experimental results, market trends) from Microsoft Fabric with unstructured documents (e.g., research reports, engineering notes) in SharePoint, then validate these findings against external references (e.g., Bing) and a high-quality “ground truth” internal knowledge stores (e.g., Azure AI Search).

This was a classic Retrieval-Augmented Generation (RAG) scenario—multiple data sources must be queried in real time and cross-checked for consistency. However, by leveraging Azure AI agent services (an agetic enterprise-ready microservices approach) alongside frameworks like Semantic Kernel, we can evolve beyond basic RAG into a mostly autonomous, agentic system. In this design, Agentic RAG and the Reflection Pattern enable each agent to iteratively refine its output until it’s confident in delivering a high-quality, validated answer—paving the way for intelligent automation that continually learns and improves.

To summarize, you’re not only bringing data to the AI but also bringing AI to the data, thus maximizing the value of your knowledge stores. By leveraging state-of-the-art retrieval solutions like Azure AI Search, while also tapping sources such as SharePoint (unstructured data) and Fabric (structured data), you can harness your most valuable asset—data—to achieve new levels of insight and automation. 

**In this demo, we have two Azure AI Agents (extending beyond a single-agent architecture):**

+ DataRetrievalAgent: Has access to Microsoft Fabric (for structured data) and SharePoint (for unstructured documents). Its job is to gather relevant internal data: for example, “failure rates of Material X in high-temperature tests,” or “engineering notes on prior tests.”

- ValidationInsightsAgent Has access to Bing / Azure Cognitive Search for external references and can run a “reflection” or “validation” step. Its job is to cross-check what was returned by the first agent and highlight missing or conflicting information. ValidationInsightsAgent has access to highly curated knowledge sources (e.g., Azure AI Search) for validating the accuracy or truthfulness of the information it receives from DataRetrievalAgent.


**Moving from a single-agent setup to a multi-agent system is now simpler than ever with Semantic Kernel. The general flow looks like this:**

1. The user asks a question (e.g., “Retrieve historical failure rates for Material X in extreme temperatures and cross-check if new standards or conflicting data exist.”).
2. The DataRetrievalAgent fetches structured data from Fabric (e.g., lab test results, analytics) and unstructured docs from SharePoint (e.g., research memos, engineering notes).
3. The ValidationInsightsAgent then queries Bing/Azure Search to verify or supplement the results. Employ a reflection pattern, where it iterates over the combined results, looking for gaps or inconsistencies. If needed, it loops back to the DataRetrievalAgent for clarifications or additional data.

Finally, the user receives a validated, summarized answer that merges internal data with external cross-checks. Thanks to the agents’ back-and-forth reflection.

### **Why This Matters**

+ **Reduced Manual Research**: Instead of manually sifting through multiple data silos and external search engines, the AI Agents automate data gathering and vetting.
+ **Higher Confidence**: Validation ensures data accuracy and highlights missing pieces, improving R&D decision-making.
+ **Enterprise-Grade Security**: Each agent can enforce On-Behalf-Of (OBO) authentication to protect sensitive data (e.g., only pulling data the user is authorized to see).
In the Jupyter Notebook

When you run the code in this Jupyter notebook:

(TODO)

In [1]:
import os
import re
import time
import logging
import json
from datetime import datetime as pydatetime
from typing import Any, List, Dict, Optional
from dotenv import load_dotenv
import asyncio
from datetime import timedelta

# Azure AI Projects
from azure.identity.aio import DefaultAzureCredential
from azure.core.exceptions import HttpResponseError

# semantic kernel
from semantic_kernel.contents import AuthorRole
from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions

# Load environment variables from .env file
load_dotenv()

# configure logging
from utils.ml_logging import get_logger

logger = get_logger()

# set the directory to the location of the script
try:
    target_directory = os.getenv("TARGET_DIRECTORY", os.getcwd())  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

### **Create Client and Load Azure AI Foundry**

Here, we initialize the Azure AI client using DefaultAzureCredential. This allows us to authenticate and connect to the Azure AI service.


In [2]:
project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential())

### **1. Creating Azure AI Agents: DataRetrievalAgent**

The DataRetrievalAgent is responsible for internal data retrieval, combining structured data from Microsoft Fabric with unstructured documents from SharePoint. This agent ensures that research teams can efficiently access critical R&D insights, such as historical failure rates, experimental results, and engineering notes—all while maintaining secure and authorized access controls.

Agent Capabilities
+ ✅ Structured Data Retrieval → Queries Microsoft Fabric for experiment logs, test results, and structured analytics.
+ ✅ Unstructured Document Search → Fetches relevant reports, blueprints, and research notes from SharePoint.
+ ✅ OBO Authentication → Uses On-Behalf-Of (OBO) authentication to ensure users can only access data they are permitted to view.

For a detailed breakdown of how to create a single Azure AI Agent and configure its data (tools) connections, please refer to:
📌 [01-single-agents-with-azure-ai-agents.ipynb](01-single-agents-with-azure-ai-agents.ipynb).

In [3]:
from azure.core.exceptions import ServiceRequestError
from azure.ai.projects.aio import AIProjectClient

async def get_connection_id(client: AIProjectClient, env_var: str) -> Optional[str]:
    """
    Retrieves the connection object using a connection name stored in an environment variable.

    Args:
        client: The Azure AI Project client.
        env_var (str): The environment variable holding the connection name.

    Returns:
        Connection object if found, otherwise raises an error.
    """
    connection_name = os.getenv(env_var)
    if not connection_name:
        logger.error(f"Missing environment variable: '{env_var}'")
        raise ValueError(f"Environment variable '{env_var}' is required.")

    try:
        connection = await client.connections.get(connection_name=connection_name)
        logger.info(f"Retrieved Connection ID for {env_var}: {connection.id}")
        return connection
    except Exception as e:
        logger.error(f"Failed to retrieve connection for {env_var}: {e}")
        raise

In [6]:
from azure.ai.projects.models import (
    SharepointTool,
    FabricTool,
    ToolSet,
)

# Initialize Azure AI Agent settings
dataretrievalagent_settings = AzureAIAgentSettings.create()

toolset = ToolSet()

try:
    # Retrieve and add SharePoint Tool
    sharepoint_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_SHAREPOINT")
    toolset.add(SharepointTool(connection_id=sharepoint_connection.id))

    # # Retrieve and add Fabric Tool
    # fabric_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_FABRIC")
    # toolset.add(FabricTool(connection_id=fabric_connection.id))

    logger.info("Successfully created ToolSet with SharePoint and Fabric tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

2025-03-20 15:38:25,007 - micro - MainProcess - INFO     Retrieved Connection ID for TOOL_CONNECTION_NAME_SHAREPOINT: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/ContosoAgentDemoSharepoint (2375541312.py:get_connection_id:22)
INFO:micro:Retrieved Connection ID for TOOL_CONNECTION_NAME_SHAREPOINT: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/ContosoAgentDemoSharepoint
2025-03-20 15:38:25,010 - micro - MainProcess - INFO     Successfully created ToolSet with SharePoint and Fabric tools. (2239274105.py:<module>:21)
INFO:micro:Successfully created ToolSet with SharePoint and Fabric tools.


In [7]:
dataretrievalagent_settings_definition = await project_client.agents.create_agent(
    model=dataretrievalagent_settings.model_deployment_name,
    name="DataRetrievalAgent",
    description=(
        "An AI agent specialized in retrieving and integrating data from two key internal sources: "
        "Microsoft Fabric for structured metrics (e.g., performance data, experiment results, and analytics for Product A and Product B) "
        "and SharePoint for unstructured documents (e.g., research papers, design reports, and qualitative insights)."
    ),
    instructions=(
        "### Role & Objective\n"
        "You are a research-focused AI assistant responsible for retrieving and integrating data from internal enterprise sources. "
        "Your responses must be precise, well-referenced, and relevant to the request.\n\n"
        
        "### Data Retrieval & Prioritization\n"
        "1. **Structured Data (Microsoft Fabric) and any questions related to clinical glucose monitoring studies:** \n"
        "   - Retrieve from Microsoft Fabric when the query involves numerical metrics, performance statistics, or structured analytics.\n"
        "   - Data is available to support comparisons of the accuracy and reliability of two glucose monitoring products (Product A and Product B). "
        "This structured data evaluates performance across different glucose ranges and includes MARD percentages, accuracy within ±20 mg/dL/±20%, and the number of readings for each product.\n"
        "   - Example: clinical glucose monitoring studies between Product A or Product B.\n\n"
        
        "2. **Qualitative Data (SharePoint):** \n"
        "   - Retrieve from SharePoint when the request pertains to qualitative insights, research papers, design documents, or textual analysis.\n"
        "   - Example: Research reports, design blueprints, or qualitative R&D insights.\n\n"
        
        "3. **Integrated Queries:** \n"
        "   - If the query requires both structured and qualitative data, retrieve from both sources and integrate the results for a comprehensive response.\n"
        "   - Ensure clarity in presenting combined insights.\n\n"
        
        "### Response Quality\n"
        "1. **Accuracy & Relevance:** Always prioritize retrieving the most current and applicable data.\n"
        "2. **Clarity & Transparency:** Clearly indicate the data source(s) used and any limitations in the available information.\n"
        "3. **Professionalism:** Present findings in a structured and concise manner to facilitate decision-making.\n"
    ),
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Internal Data Retrieval for R&D",
        "data_source": "Microsoft Fabric (structured metrics) and SharePoint (research documents)",
        "response_validation": "Ensure data accuracy and relevance by retrieving the most recent and applicable data."
    },
)

# Print the agent's run ID (agent ID)
print(f"DataRetrievalAgent Run ID: {dataretrievalagent_settings_definition.id}")


DataRetrievalAgent Run ID: asst_BN7AvpIntZuA4s7nI2wQB4ri


In [8]:
dataretrievalagent = AzureAIAgent(
    client=project_client,
    definition=dataretrievalagent_settings_definition,
    polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
)

thread = await project_client.agents.create_thread()

USER_INPUTS = [
    "How does Product A compare to Product B in terms of MARD percentage across different glucose ranges?"
    "What are the latest trends in R&D?",
    "What are the key features from Dexcom G7 CGM System?",
   ]
try:
    for user_input in USER_INPUTS:
        # Add the user input as a chat message
        await dataretrievalagent.add_chat_message(thread_id=thread.id, message=user_input)
        print(f"👤 **User:** {user_input}\n")
        
        # Invoke the agent for the specified thread and stream the response
        async for content in dataretrievalagent.invoke(thread_id=thread.id):
            # Only print non-tool messages
            if content.role != AuthorRole.TOOL:
                print(f"🤖 **Agent:** {content.content}\n")
                
except HttpResponseError as e:
    try:
        error_json = json.loads(e.response.content)
        logging.error(f"❌ **Error Message:** {error_json.get('Message')}")
    except json.JSONDecodeError:
        logging.error(f"❌ **Non-JSON Error Content:** {e.response.content}")

👤 **User:** How does Product A compare to Product B in terms of MARD percentage across different glucose ranges?What are the latest trends in R&D?

🤖 **Agent:** ### MARD Percentage Comparison

For Product A, the Mean Absolute Relative Difference (MARD) was reported as 8.7% (±0.4%) in trials under various glucose ranges【3:2†source】. Unfortunately, specific MARD data for Product B across different glucose ranges wasn't provided in the available documents. Further detailed studies would be needed to provide a direct comparison for Product B.

### Latest Trends in R&D

1. **Non-Invasive or Minimally Invasive Sensors**: R&D is exploring optical spectroscopy and radiofrequency monitoring for glucose measurement, aiming to reduce skin insertions and adhesive issues【5:1†source】.

2. **Extended Sensor Lifespan**: Future CGM devices may last for several months, reducing the need for frequent replacements while maintaining accuracy【5:1†source】.

3. **Enhanced Interoperability**: There is a push f

### **2. Creating Azure AI Agents: ValidationInsightsAgent**

The ValidationInsightsAgent is designed to validate and cross-check the data retrieved by the DataRetrievalAgent. It accesses external references such as Bing and Azure Cognitive Search to verify and supplement the internal data. Additionally, it leverages highly curated knowledge sources (e.g., Azure AI Search) to ensure the accuracy and truthfulness of the information, using a reflection or validation step to highlight missing or conflicting details.

Agent Capabilities:

+ ✅ External Reference Verification → Queries Bing and Azure Cognitive Search for real-time validation.
+ ✅ Reflection & Validation Step → Iteratively reviews and refines the information received from the DataRetrievalAgent.
+ ✅ Curated Knowledge Validation → Uses Azure AI Search to confirm the accuracy and reliability of internal data.

For a detailed breakdown of how to create a ValidationInsightsAgent and configure its external tools and connections, please refer to:
📌 [01-single-agents-with-azure-ai-agents.ipynb](01-single-agents-with-azure-ai-agents.ipynb).


In [10]:
from azure.ai.projects.models import (
    BingGroundingTool,
    AzureAISearchTool,
    FileSearchTool,
    VectorStore,
    OpenAIFile
)

# Initialize Azure AI Agent settings
validationinsightagent_settings = AzureAIAgentSettings.create()

# Create a ToolSet to manage tools
toolset = ToolSet()

try:
    # Retrieve and add the Bing Grounding Tool
    bing_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_BING")
    toolset.add(BingGroundingTool(connection_id=bing_connection.id))
    logger.info("Bing Grounding Tool added successfully.")

    # Retrieve and add the Azure AI Search Tool
    # search_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_SEARCH")
    # azure_ai_search_connection = AzureAISearchTool(
    #     index_connection_id=search_connection.id,
    #     index_name="ai-agentic-index"
    # )
    # toolset.add(azure_ai_search_connection)
    # logger.info("Azure AI Search Tool added successfully.")

    # Dynamically construct the PDF file path using os.path.join
    pdf_file_path = os.path.join(target_directory, "data", "product_data", "ProductATechncialArchitecture.pdf")
    logger.info(f"Using PDF file path: {pdf_file_path}")

    file: OpenAIFile = await project_client.agents.upload_file_and_poll(file_path=pdf_file_path, purpose="assistants")
    vector_store: VectorStore = await project_client.agents.create_vector_store_and_poll(
        file_ids=[file.id], name="my_vectorstore"
    )

    # 2. Create file search tool with uploaded resources
    file_search = FileSearchTool(vector_store_ids=[vector_store.id])

    toolset.add(file_search)
    logger.info("Azure AI Search Tool added successfully.")
 

    logger.info("Successfully created ToolSet with Bing and File Search tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

2025-03-20 15:40:22,801 - micro - MainProcess - INFO     Retrieved Connection ID for TOOL_CONNECTION_NAME_BING: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/agentsbinggrounding (2375541312.py:get_connection_id:22)
INFO:micro:Retrieved Connection ID for TOOL_CONNECTION_NAME_BING: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/agentsbinggrounding
2025-03-20 15:40:22,808 - micro - MainProcess - INFO     Bing Grounding Tool added successfully. (1615353572.py:<module>:19)
INFO:micro:Bing Grounding Tool added successfully.
2025-03-20 15:40:22,813 - micro - MainProcess - INFO     Using PDF file path: c:\Users\pablosal\Desktop\azure-ai-agent-services-demo\data\product_data\ProductATechncialArchitecture.pdf (1615353572.py:<module>:32)
INFO:micro:Using P

In [11]:
# Create or update a new Validation Insights Agent
validationinsightsagent_definition = await project_client.agents.create_agent(
    model=dataretrievalagent_settings.model_deployment_name,
    name="ValidationInsightsAgent",
    description=(
        "An AI agent designed to validate and refine R&D insights by cross-checking "
        "both internal enterprise data sources (e.g., file search vector store, SharePoint, Fabric) "
        "and external public data (Bing, Azure AI Search). It uses a reflection pattern "
        "to ensure response accuracy, consistency, and proper citations."
    ),
    instructions=(
        "### Role & Objective\n"
        "You are the 'Validation Insights' AI assistant, responsible for validating insights by "
        "retrieving and cross-checking data from internal enterprise sources and external web searches. "
        "Your responses must be accurate, well-referenced, and actionable."
        "\n\n"
        "### Data Retrieval & Prioritization\n"
        "1. **Primary Data Source**: Always consult internal enterprise sources first (File Search Vector Store). "
        "2. **External Validation**: If internal data is insufficient or requires verification, utilize Bing and Azure AI Search."
        "\n\n"
        "### Reflection & Validation\n"
        "1. **Evaluate Consistency**: After retrieving data, compare sources to identify inconsistencies.\n"
        "2. **Perform Refinement**: If contradictions or gaps exist, conduct a second retrieval pass to ensure accuracy."
        "\n\n"
        "### Response Quality\n"
        "1. **Accuracy & Transparency**: Always prioritize factual correctness and completeness in responses.\n"
        "2. **Citations & References**: Clearly cite the sources (internal or external) used in your response.\n"
        "3. **Clarity & Professionalism**: Deliver insights concisely and in a structured format. Be transparent if data validation is pending."
    ),
    toolset=toolset,
    # Prefer the internal file search vector store by default
    tool_resources=file_search.resources,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Cross-Validation and Insight Generation for R&D",
        "data_source": "Internal (Fabric, SharePoint) and External (Bing, Azure AI Search)",
        "response_validation": (
            "Employ a reflection step to cross-check data accuracy "
            "and provide clear citations in final responses."
        )
    },
)

print(f"ValidationInsightsAgent Run ID: {validationinsightsagent_definition.id}")


ValidationInsightsAgent Run ID: asst_cflIDuNZR0fOpTJp59zhhDdn


In [12]:
validation_agent = AzureAIAgent(
    client=project_client,
    definition=validationinsightsagent_definition,
    polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
)

# Create a conversation thread for the agent
thread = await project_client.agents.create_thread()

# Define a list of user inputs designed to trigger both internal search (e.g., product architecture) 
# and external market trend validation.
USER_INPUTS = [
    "What are the characteristics and architecture of Product A?",
    "What are the key features from Dexcom G7 CGM System?",
]

try:
    for user_input in USER_INPUTS:
        # Add the user input as a chat message
        await validation_agent.add_chat_message(thread_id=thread.id, message=user_input)
        print(f"👤 **User:** {user_input}\n")
        
        # Invoke the agent for the specified thread and stream the response
        async for content in validation_agent.invoke(thread_id=thread.id):
            # Only print non-tool messages
            if content.role != AuthorRole.TOOL:
                print(f"🤖 **Agent:** {content.content}\n")
                
except HttpResponseError as e:
    try:
        error_json = json.loads(e.response.content)
        logger.error(f"❌ **Error Message:** {error_json.get('Message')}")
    except json.JSONDecodeError:
        logger.error(f"❌ **Non-JSON Error Content:** {e.response.content}")

👤 **User:** What are the characteristics and architecture of Product A?

🤖 **Agent:** ### Characteristics and Architecture of Product A

**Characteristics:**

1. **Functionality**: Product A is an integrated continuous glucose monitoring (iCGM) system designed to replace traditional blood glucose monitoring by providing real-time interstitial glucose readings【4:0†source】.

2. **Accuracy**: Demonstrates a high level of accuracy with a Mean Absolute Relative Difference (MARD) of approximately 8.5% to 9.2% during varying glucose conditions【4:1†source】.

3. **Usability**: User-friendly with a mobile application that provides real-time data, customizable alerts, and compatibility with other diabetes management devices【4:4†source】.

4. **Durability**: Engineered for long-term sensor wear (up to 10 days) with a hypoallergenic adhesive patch【4:4†source】.

5. **Adaptability**: Includes a future-ready adaptive machine learning algorithm for personalized insulin dosing and lifestyle adjustments【4

### **2. Creating Multi Agent System**

The following sample demonstrates how to create an OpenAI assistant using either Azure OpenAI or OpenAI, a chat completion agent and have them participate in a group chat to work towards the user's requirement.


In [16]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

In [18]:
import os
import asyncio
import json
from typing import List, Dict, Optional

from azure.identity.aio import DefaultAzureCredential
from azure.core.exceptions import HttpResponseError
from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.azure_ai import AzureAIAgent
from semantic_kernel.agents.strategies import TerminationStrategy, KernelFunctionSelectionStrategy
from semantic_kernel.functions import KernelFunctionFromPrompt


##############################################################################
# 2) A Custom Termination Strategy
###############################################################################
class ApprovalTerminationStrategy(TerminationStrategy):
    """
    Ends the conversation if the Evaluator agent's last output includes the word 'approved'.
    """
    def __init__(self, agents, maximum_iterations=10):
        super().__init__(maximum_iterations=maximum_iterations)
        self.agents = agents

    async def should_agent_terminate(self, agent, history) -> bool:
        # We assume the Evaluator is among self.agents
        # If the final content from the Evaluator includes 'approved', we end
        last_msg = history[-1]
        return (last_msg.name in [a.name for a in self.agents]) and ("approved" in last_msg.content.lower())

###############################################################################
# 3) Creating a Kernel for the Selection Function
###############################################################################
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    """
    Creates a Semantic Kernel instance with an Azure OpenAI chat completion service.
    Make sure you have environment variables for your Azure OpenAI keys/endpoint.
    """
    from semantic_kernel import Kernel
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    kernel = Kernel()
    # Retrieve environment vars (adjust to your naming)
    AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
    AZURE_OPENAI_API_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
    AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
    AZURE_AOAI_CHAT_MODEL_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID")  

    # Register an Azure Chat Completion service in the kernel
    kernel.add_service(
        service=AzureChatCompletion(
            deployment_name=AZURE_AOAI_CHAT_MODEL_DEPLOYMENT,
            api_key=AZURE_OPENAI_KEY,
            endpoint=AZURE_OPENAI_API_ENDPOINT,
            api_version=AZURE_OPENAI_API_VERSION,
        )
    )
    return kernel

###############################################################################
# 4) Example LLM Prompt for Multi-Agent Turn Selection
###############################################################################
RETRIEVER_NAME = "ValidationInsightsAgent"
EVALUATOR_NAME = "DataRetrievalAgent"

selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
Determine which participant takes the next turn in a conversation based on the most recent participant.
State only the name of the participant to take the next turn.
No participant should take more than one turn in a row.

Choose only from these participants:
- {RETRIEVER_NAME}
- {EVALUATOR_NAME}

Always follow these rules when selecting the next participant:
- {RETRIEVER_NAME} retrieving the document or relevant data the query.
- After {RETRIEVER_NAME}, it is {EVALUATOR_NAME}'s turn to evaluate the content.
- After {EVALUATOR_NAME}, the workflow may terminate if 'approved'.

History:
{{{{$history}}}}
""",
)

###############################################################################
# 5) Utility Logging For Agent Interaction
###############################################################################
def log_agent_invocation(agent_name: str, role: str, input_data: str, output_data: str):
    trace_data = {
        "agent_name": agent_name,
        "role": role,
        "input": input_data,
        "output": output_data,
    }
    logger.info("AGENT INVOCATION:\n" + json.dumps(trace_data, indent=4))

###############################################################################
# 6) Main Async Function
###############################################################################
async def main():
    # 2. Initialize the credential & AzureAIAgent client
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        # Pre-existing Azure AI Agents (IDs from your environment)
        ValidationInsightsAgentID = "asst_cflIDuNZR0fOpTJp59zhhDdn"
        DataRetrievalAgentID = "asst_BN7AvpIntZuA4s7nI2wQB4ri"

        # 3. Retrieve each agent definition from your Azure AI Agent Service
        #    Assume the Formulator & Evaluator are the same agent or different? 
        #    Below, just demonstrating you might have 2 definitions for 3 roles.
        dataretrieval_def = await client.agents.get_agent(agent_id=DataRetrievalAgentID)
        validation_def = await client.agents.get_agent(agent_id=ValidationInsightsAgentID)

        # 4. Build SK Agents from these definitions
        agent_retriever = AzureAIAgent(client=client, definition=dataretrieval_def)
        agent_evaluator = AzureAIAgent(client=client, definition=validation_def)

        # 5. Setup the multi-agent chat
        chat = AgentGroupChat(
            agents=[agent_retriever, agent_evaluator],
            termination_strategy=ApprovalTerminationStrategy(
                maximum_iterations=10,
                agents=[agent_evaluator],  # The Evaluator decides final approval
            ),
            selection_strategy=KernelFunctionSelectionStrategy(
                function=selection_function,
                kernel=_create_kernel_with_chat_completion("selection"),
                # We parse the output of the LLM prompt to choose next agent
                result_parser=lambda result: str(result.value[0]) if result.value else EVALUATOR_NAME,
                agent_variable_name="agents",   # the variable in the prompt
                history_variable_name="history", # the variable in the prompt
            ),
        )

        # Example user system or user message
        system_message = """
        You are orchestrating a multi-step R&D conversation between two roles: 
        1) Retriever – gathers internal data from Fabric/SharePoint and external references from Bing/Azure Search.
        2) Evaluator – cross-validates the retrieved data, identifies missing insights, and decides if it is 'approved' or if more data is needed.

        **Demo Flow: AI-Powered R&D and Product Design**
        1. A researcher initiates a query for internal experiment data and external market trends to refine a new product design.
        2. Retriever queries Fabric AI Skill for structured results (e.g., experiment data), and SharePoint for related research papers, engineering reports, and design notes.
        3. Evaluator checks the alignment between internal test results and external market or regulatory feedback. Identifies any conflicting standards or data gaps. If needed, it reruns queries for additional sources.
        4. The system compiles an AI-generated report summarizing key findings, highlighting risk factors, data gaps, and actionable recommendations.
        5. The researcher can ask follow-up questions about materials, regulatory risks, or design improvements. 

        The user can keep asking questions until the Evaluator decides it is 'approved' (final answer) or more data is required.
        """
        await chat.add_chat_message(message=system_message)
        logger.info(f"System Setup: {system_message}")

        # Updated user message introducing the high-level request
        user_message = """
        What are the characteristics and architecture of Product A? Please research the market for similar products and gather insights from our research team. What other products have we done competitor analysis on?
        """
        await chat.add_chat_message(message=user_message)
        logger.info(f"User Input: {user_message}")

        # 6. Real-time streaming of conversation
        try:
            last_input = user_message
            async for content in chat.invoke():
                agent_name = content.name or "Unknown"
                agent_role = content.role.name
                agent_output = content.content

                # Log invocation
                log_agent_invocation(agent_name, agent_role, last_input, agent_output)

                # Print
                print(f"{agent_role} - {agent_name}: {agent_output}")

                last_input = agent_output

            logger.info(f"Workflow Complete? {chat.is_complete}")

        except HttpResponseError as err:
            logger.error(f"Error while streaming conversation: {err}")

        finally:
            # 7. Optionally reset or cleanup if ephemeral
            await chat.reset()
            # If you'd like to delete ephemeral agent definitions, do so here
            # e.g., await client.agents.delete_agent(DataRetrievalAgentID)
            #       await client.agents.delete_agent(ValidationInsightsAgentID)



In [19]:
await main() 

2025-03-20 15:43:35,634 - micro - MainProcess - INFO     System Setup: 
        You are orchestrating a multi-step R&D conversation between two roles: 
        1) Retriever – gathers internal data from Fabric/SharePoint and external references from Bing/Azure Search.
        2) Evaluator – cross-validates the retrieved data, identifies missing insights, and decides if it is 'approved' or if more data is needed.

        **Demo Flow: AI-Powered R&D and Product Design**
        1. A researcher initiates a query for internal experiment data and external market trends to refine a new product design.
        2. Retriever queries Fabric AI Skill for structured results (e.g., experiment data), and SharePoint for related research papers, engineering reports, and design notes.
        3. Evaluator checks the alignment between internal test results and external market or regulatory feedback. Identifies any conflicting standards or data gaps. If needed, it reruns queries for additional sources.
 

ASSISTANT - ValidationInsightsAgent: ### Characteristics and Architecture of Product A

**Product A** is an advanced integrated continuous glucose monitoring (iCGM) system. Here are its key characteristics and architecture:

1. **Sensor Module:**
   - **Sensor Core:** Factory-calibrated electrochemical sensor with high sensitivity, measuring interstitial glucose every 5 minutes.
   - **Protective Membrane:** Biocompatible membrane minimizes interference, ensuring signal stability.
   - **Adhesive and Patch:** Hypoallergenic, durable for up to 10 days, with overpatch for durability during activities【5:0†source】.

2. **Transmitter and Communication Layer:**
   - **Transmitter:** Low-power, battery-powered, converting signals every 5 minutes, with robust battery life.
   - **Connectivity:** Secure Bluetooth, AES-256 encryption, with data buffering for up to 24 hours【5:4†source】.

3. **Software and Analytics:**
   - **Mobile Application:** Real-time data display, trend analysis, and custom

2025-03-20 15:44:40,554 - micro - MainProcess - INFO     AGENT INVOCATION:
{
    "agent_name": "ValidationInsightsAgent",
    "role": "ASSISTANT",
    "input": "### Characteristics and Architecture of Product A\n\n**Product A** is an advanced integrated continuous glucose monitoring (iCGM) system. Here are its key characteristics and architecture:\n\n1. **Sensor Module:**\n   - **Sensor Core:** Factory-calibrated electrochemical sensor with high sensitivity, measuring interstitial glucose every 5 minutes.\n   - **Protective Membrane:** Biocompatible membrane minimizes interference, ensuring signal stability.\n   - **Adhesive and Patch:** Hypoallergenic, durable for up to 10 days, with overpatch for durability during activities\u30105:0\u2020source\u3011.\n\n2. **Transmitter and Communication Layer:**\n   - **Transmitter:** Low-power, battery-powered, converting signals every 5 minutes, with robust battery life.\n   - **Connectivity:** Secure Bluetooth, AES-256 encryption, with data buf

ASSISTANT - ValidationInsightsAgent: ### Characteristics and Architecture of Product A

**Product A** is a sophisticated integrated continuous glucose monitoring (iCGM) system with the following features:

1. **Sensor Module:**
   - **Sensor Core:** Factory-calibrated electrochemical sensor for real-time glucose measurement.
   - **Protective Membrane:** Biocompatible to reduce interference.
   - **Adhesive and Patch:** Hypoallergenic and durable for up to 10 days【5:0†source】.

2. **Transmitter and Communication Layer:**
   - **Transmitter:** Low-power, battery-powered for signal conversion.
   - **Connectivity:** Secure Bluetooth with AES-256 encryption【5:4†source】.

3. **Software and Analytics:**
   - **Mobile Application:** Real-time data and alerts.
   - **Cloud-Based Management:** Advanced analytics and personalized insights【5:4†source】.

### Internal Competitor Analysis

The company has conducted competitor analysis on several products, though specific names are not detailed in t

2025-03-20 15:44:48,734 - micro - MainProcess - INFO     AGENT INVOCATION:
{
    "agent_name": "DataRetrievalAgent",
    "role": "ASSISTANT",
    "input": "### Characteristics and Architecture of Product A\n\n**Product A** is a sophisticated integrated continuous glucose monitoring (iCGM) system with the following features:\n\n1. **Sensor Module:**\n   - **Sensor Core:** Factory-calibrated electrochemical sensor for real-time glucose measurement.\n   - **Protective Membrane:** Biocompatible to reduce interference.\n   - **Adhesive and Patch:** Hypoallergenic and durable for up to 10 days\u30105:0\u2020source\u3011.\n\n2. **Transmitter and Communication Layer:**\n   - **Transmitter:** Low-power, battery-powered for signal conversion.\n   - **Connectivity:** Secure Bluetooth with AES-256 encryption\u30105:4\u2020source\u3011.\n\n3. **Software and Analytics:**\n   - **Mobile Application:** Real-time data and alerts.\n   - **Cloud-Based Management:** Advanced analytics and personalized ins

...

2025-03-20 15:45:46,759 - micro - MainProcess - INFO     AGENT INVOCATION:
{
    "agent_name": "ValidationInsightsAgent",
    "role": "ASSISTANT",
    "input": "If you have more questions or need additional insights about Product A or market trends, feel free to ask!",
    "output": "Is there anything else you'd like to explore about Product A or market trends?"
} (1936533635.py:log_agent_invocation:98)
INFO:micro:AGENT INVOCATION:
{
    "agent_name": "ValidationInsightsAgent",
    "role": "ASSISTANT",
    "input": "If you have more questions or need additional insights about Product A or market trends, feel free to ask!",
    "output": "Is there anything else you'd like to explore about Product A or market trends?"
}


ASSISTANT - ValidationInsightsAgent: Is there anything else you'd like to explore about Product A or market trends?


2025-03-20 15:45:56,581 - micro - MainProcess - INFO     AGENT INVOCATION:
{
    "agent_name": "DataRetrievalAgent",
    "role": "ASSISTANT",
    "input": "Is there anything else you'd like to explore about Product A or market trends?",
    "output": "Would you like more information on Product A or further market analysis? Let me know how I can assist!"
} (1936533635.py:log_agent_invocation:98)
INFO:micro:AGENT INVOCATION:
{
    "agent_name": "DataRetrievalAgent",
    "role": "ASSISTANT",
    "input": "Is there anything else you'd like to explore about Product A or market trends?",
    "output": "Would you like more information on Product A or further market analysis? Let me know how I can assist!"
}


ASSISTANT - DataRetrievalAgent: Would you like more information on Product A or further market analysis? Let me know how I can assist!


CancelledError: 

## Creting Streamlit App 

In [ ]:
# 2. Import Necessary Modules
# Import the required modules in your Python script:

import os
import asyncio
import streamlit as st
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.azure_ai import AzureAIAgent
from semantic_kernel.agents.strategies import TerminationStrategy, KernelFunctionSelectionStrategy
from semantic_kernel.functions import KernelFunctionFromPrompt


In [ ]:
# 3. Define the Approval Termination Strategy
# Create a custom termination strategy to end the conversation based on specific criteria:

class ApprovalTerminationStrategy(TerminationStrategy):
    def __init__(self, agents, maximum_iterations=10):
        super().__init__(maximum_iterations=maximum_iterations)
        self.agents = agents

    async def should_agent_terminate(self, agent, history) -> bool:
        last_msg = history[-1]
        return (last_msg.name in [a.name for a in self.agents]) and ("approved" in last_msg.content.lower())


In [ ]:
# 4. Initialize the Kernel with Chat Completion
# Set up the Semantic Kernel with an Azure OpenAI chat completion service:

def _create_kernel_with_chat_completion() -> Kernel:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    kernel = Kernel()
    AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
    AZURE_OPENAI_API_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
    AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
    AZURE_AOAI_CHAT_MODEL_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_MODEL_DEPLOYMENT", "gpt-4")

    kernel.add_service(
        service=AzureChatCompletion(
            deployment_name=AZURE_AOAI_CHAT_MODEL_DEPLOYMENT,
            api_key=AZURE_OPENAI_KEY,
            endpoint=AZURE_OPENAI_API_ENDPOINT,
            api_version=AZURE_OPENAI_API_VERSION,
        )
    )
    return kernel


In [ ]:
# 5. Define the Selection Function Prompt
# Create a prompt to determine the next participant in the conversation:

RETRIEVER_NAME = "ValidationInsightsAgent"
EVALUATOR_NAME = "DataRetrievalAgent"

selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
Determine which participant takes the next turn in a conversation based on the most recent participant.
State only the name of the participant to take the next turn.
No participant should take more than one turn in a row.

Choose only from these participants:
- {RETRIEVER_NAME}
- {EVALUATOR_NAME}

Always follow these rules when selecting the next participant:
- {RETRIEVER_NAME} retrieves the document or relevant data for the query.
- After {RETRIEVER_NAME}, it is {EVALUATOR_NAME}'s turn to evaluate the content.
- After {EVALUATOR_NAME}, the workflow may terminate if 'approved'.

History:
{{{{$history}}}}
""",
)


In [ ]:
# 6. Initialize Agents and Chat
# Set up the agents and the chat interface:

async def initialize_agents():
    creds = DefaultAzureCredential()
    client = await AzureAIAgent.create_client(credential=creds)

    ValidationInsightsAgentID = "asst_kdFT72VdYH0YpoG3tJ5lmoFy"
    DataRetrievalAgentID = "asst_Wo0GJ9MpmvkfRPNwllC7bYFS"

    dataretrieval_def = await client.agents.get_agent(agent_id=DataRetrievalAgentID)
    validation_def = await client.agents.get_agent(agent_id=ValidationInsightsAgentID)

    agent_retriever = AzureAIAgent(client=client, definition=dataretrieval_def)
    agent_evaluator = AzureAIAgent(client=client, definition=validation_def)

    chat = AgentGroupChat(
        agents=[agent_retriever, agent_evaluator],
        termination_strategy=ApprovalTerminationStrategy(
            maximum_iterations=10,
            agents=[agent_evaluator],
        ),
        selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=_create_kernel_with_chat_completion(),
            result_parser=lambda result: str(result.value[0]) if result.value else EVALUATOR_NAME,
            agent_variable_name="agents",
            history_variable_name="history",
        ),
    )

    return chat


In [ ]:
# 7. Build the Streamlit Interface
# Create the Streamlit interface to interact with the agents:

async def main():
    st.title("Multi-Agent Chat Interface")

    if "chat" not in st.session_state:
        st.session_state.chat = await initialize_agents()
        st.session_state.history = []

    user_input = st.chat_input("Enter your message:")
    if user_input:
        st.session_state.history.append({"role": "user", "content": user_input})
        await st.session_state.chat.add_chat_message(message=user_input)

        async for content in st.session_state.chat.invoke():
            agent_name = content.name or "Unknown"
            agent_role = content.role.name
            agent_output = content.content

            st.session_state.history.append({"role": agent_role, "name": agent_name, "content": agent_output})

            st.chat_message(agent_role).write(f"**{agent_name}:** {agent_output}")

        if st.session_state.chat.is_complete:
            st.write("Conversation has been approved and terminated.")
            st.session_state.chat.reset()
            st.session_state.history = []

if __name__ == "__main__":
    asyncio.run(main())


8. Run the Streamlit Application

Save your script (e.g., app.py) and run it using Streamlit:

In [1]:
!streamlit run src/chatapp/app.py


^C
